In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import cv2
import re

mpl.rcParams['font.sans-serif'] = ['SimHei']
%matplotlib inline

for lib in [('numpy',np), ('pandas', pd), ('matplotlib', mpl), ('opencv', cv2)]:
    print(f'{lib[0]:>10} version is {lib[1].__version__}')

numpy version is 1.17.2
    pandas version is 0.25.1
matplotlib version is 3.1.1
    opencv version is 3.4.1


In [2]:
def test_train_file_names(k, d):
    """
    将指定目录下的文件分组，每组随机提取 k 比率的文件作为训练集，其余的作为测试集。
    注意：该函数只对文件名操作，并非实际对文件操作
    
    Paramaters
    -----------
    k -- 训练集所占比率 0<k<=1
    d -- 存放文件的目录
    
    Returns
    -------
    ndarray 1-D 训练集
    ndarray 1-D 测试集
    """
    # 构建的DataFrame的字段名称
    FILE_NAME = 'file_name'
    FILE_GROUP_NAME = 'file_group_name'
    # 获取文件列表
    fn_list = os.listdir(d)
    # 根据文件列表创建 DataFrame
    df = pd.DataFrame(fn_list, columns=[FILE_NAME])
    # 增加文件文组标示
    # extract 的说明: 根据正则表达式 r'(.*)(_)' 提取字符串
    #                比如: 's1_1.bmp' -> ('s1', '_')
    #                     's10_10.bmp' -> ('s10', '_')
    df[FILE_GROUP_NAME] = df.file_name.str.extract(r'(.*)(_)')[0]
    # 根据 FILE_GROUP_NAME 进行分组, 得到分组对象 grouped
    # 分组对象 grouded 的 groups 属性是一个字典, key 是分组名称, value 是分组数据(row index)
    # 形如: {'s1': [1,2,3,...], 's2': [10,11,12...]}
    grouped = df.groupby([FILE_GROUP_NAME])
    train_set = np.array([])
    test_set = np.array([])
    # 循环所有分组数据, 每次操作一组数据
    for v in grouped.groups.values():
        # 从 DF 中获取 file_name 字段 的数据
        # v 是 row index.
        data = df[FILE_NAME][v]
        # 随机获取指定比率的数据 作为训练集(文件名称)
        train = data.sample(frac=k)
        # 提取剩余数据 作为测试集(文件名称)
        test = data.drop(train.index)
        # 将一组中的随机提取的训练集数据放入总训练集中
        train_set = np.concatenate((train_set, train.values))
        # 将一组中的测试集数据放入总测试集中
        test_set = np.concatenate((test_set, test.values))
    return train_set, test_set

def img2vector(img_file_path_name):
    """
    图片向量化
    将给定的图片(m, n), 输出(1, m*n)的行向量
    """
    img = cv2.imread(img_file_path_name, 0)
    return img.flatten().reshape(1, -1)

def load_orl(k, d):
    """
    加载图片文件形成 matrix
    
    Parameters
    ----------
    k {float} -- 图片文件训练集比率 0 < k <= 1
    d {string} -- 图片文件所在目录
    
    Returns
    -------
    {ndarray 2D} -- 训练集(特征)
    {ndarray 2D} -- 训练集(标签/真实值)
    {ndarray 2D} -- 测试集(特征)
    {ndarray 2D} -- 测试集(标签/真实值)
    """
    # 图片文件拆分为 训练集用文件 测试集用文件
    train_file_names, test_file_names = test_train_file_names(k, d)
    # 生成测试集第一条数据
    train_set = img2vector(os.path.join(d, train_file_names[0]))
    # 生成测试集第一个标签
    train_label = np.array([re.sub('_.*', '', train_file_names[0])])
    # 生成训练集第一条数据
    test_set = img2vector(os.path.join(d, test_file_names[0]))
    # 生成训练集第一个标签
    test_label = np.array([re.sub('_.*', '', test_file_names[0])])
    # 循环生成训练集数据和训练集标签
    # 因为第一条数据已经生成过, 因此需要从第二条开始循环
    for f in train_file_names[1:]:
        img = img2vector(os.path.join(d, f))
        train_set = np.concatenate((train_set, img))
        train_label = np.append(train_label, re.sub('_.*', '', f))
    # 循环生成测试集数据和训练集标签
    # 因为第一条数据已经生成过, 因此需要从第二条开始循环
    for f in test_file_names[1:]:
        img = img2vector(os.path.join(d, f))
        test_set = np.concatenate((test_set, img))
        test_label = np.append(test_label, re.sub('_.*', '', f))
    return train_set, train_label, test_set, test_label

def PCA(data, r):
    """
    Principle Component Analysis (PCA) 主成分分析 算法

    Parameters
    ---------
    data {ndarray 2D} -- 原数据
    r {int} -- 保留维度

    Returns
    -------
    {ndarray 2D} -- PCA结果
    {ndarray 2D} -- 协方差矩阵
    {ndarray 1D} -- 协方差矩阵特征值
    {ndarray 2D} -- 协方差矩阵特征向量
    """
    # 原始数据的记录量，即行的数量
    rows = data.shape[0]
    # 复制一份原数据并进行转置
    # 转置后的矩阵每行代表一个特征，每列代表一条记录（一张图片）
    X = np.copy(data).T
    # 按行计算平均值
    mu = X.mean(axis=1)[:, None]
    # 按行平句话
    A = X - mu
    C = (A @ A.T) / rows
    D, V = np.linalg.eig(C)
    idx = D.argsort()[::-1]
    D = D[idx]
    V = V[:, idx]
    P = V.T
    return P, C, D, V